# Population

> Data Structures for Population

In [14]:
#| default_exp population

In [15]:
#| hide
from nbdev.showdoc import *


In [16]:
#| echo: false
from nbdev.showdoc import show_doc

In [17]:
#| export
from dataclasses import field
from typing import List, Optional, Dict, Callable

from flax.struct import dataclass as flax_dataclass # Using flax's dataclass for JAX-friendliness
import jax
import jax.numpy as jnp

from chewc.sp import SimParam
from typing import Tuple
from numpy.random import default_rng
import msprime
import tskit
import numpy as np
import random
from collections import defaultdict

#testing
import jax
import jax.numpy as jnp
from fastcore.test import test_eq, test_ne

In [18]:
#| export
# --- Core Population Structure ---



@flax_dataclass(frozen=True) # Make the class immutable, a JAX best practice
class Population:
    """
    A container for all data related to a population of individuals, designed
    for JAX-based genetic simulations.

    This structure is immutable. All operations that modify a population should
    return a new Population object.

    Attributes:
        geno (jnp.ndarray): A 4D array representing the genotypes of the population.
            Shape: `(nInd, nChr, ploidy, nLoci)`. dtype: `jnp.uint8`.

        id (jnp.ndarray): The primary, user-facing identifier for each individual.
            These IDs may not be contiguous or sorted. Shape: `(nInd,)`.
        iid (jnp.ndarray): The internal, zero-indexed, contiguous identifier.
            Crucial for robust indexing in JAX operations. Shape: `(nInd,)`.
        mother (jnp.ndarray): Array of internal IDs (`iid`) for the mother of each
            individual. A value of -1 indicates no known mother. Shape: `(nInd,)`.
        father (jnp.ndarray): Array of internal IDs (`iid`) for the father of each
            individual. A value of -1 indicates no known father. Shape: `(nInd,)`.
        sex (jnp.ndarray): The sex of each individual, represented numerically
            (e.g., 0 for male, 1 for female). dtype: `jnp.int8`. Shape: `(nInd,)`.

        pheno (jnp.ndarray): The phenotypic values for each individual.
            Shape: `(nInd, nTraits)`.
        fixEff (jnp.ndarray): The value of a fixed effect for each individual,
            often used as an intercept in genomic selection models. Shape: `(nInd,)`.
        
        bv (Optional[jnp.ndarray]): The true breeding values (additive genetic effects)
            for each individual. Shape: `(nInd, nTraits)`.
        dd (Optional[jnp.ndarray]): The true dominance deviations for each individual.
            Shape: `(nInd, nTraits)`.
        aa (Optional[jnp.ndarray]): The true additive-by-additive epistatic deviations
            for each individual. Shape: `(nInd, nTraits)`.

        ebv (Optional[jnp.ndarray]): The estimated breeding values for each
            individual. Shape: `(nInd, nTraits)`.
        gxe (Optional[jnp.ndarray]): Genotype-by-environment interaction effects.
            Shape depends on the specific GxE model.

        misc (Dict): A dictionary for storing miscellaneous, non-JAX-critical
            metadata about individuals. Static.
        miscPop (Dict): A dictionary for storing miscellaneous, non-JAX-critical
            metadata about the entire population. Static.
    """
    # --- Core Genotype Info ---
    geno: jnp.ndarray
    
    # --- Pedigree and Identifiers ---
    id: jnp.ndarray
    iid: jnp.ndarray
    mother: jnp.ndarray
    father: jnp.ndarray
    sex: jnp.ndarray

    # --- Trait and Value Data ---
    pheno: jnp.ndarray
    fixEff: jnp.ndarray
    gv: Optional[jnp.ndarray] = None      # Genetic Value (BV + Intercept)
    bv: Optional[jnp.ndarray] = None      # Breeding Value (Additive)
    dd: Optional[jnp.ndarray] = None      # Dominance Deviations
    aa: Optional[jnp.ndarray] = None      # Additive-by-Additive Epistatic Deviations
    
    ebv: Optional[jnp.ndarray] = None
    gxe: Optional[jnp.ndarray] = None

    # --- Metadata ---
    misc: Dict = field(default_factory=dict)
    miscPop: Dict = field(default_factory=dict)

    def __post_init__(self):
        """Validates the consistency of the population data after initialization."""
        n_ind = self.nInd
        # Validate pedigree and identifier shapes
        assert self.iid.shape[0] == n_ind, f"Internal ID array length ({self.iid.shape[0]}) must match number of individuals ({n_ind})."
        assert self.mother.shape[0] == n_ind, f"Mother array length ({self.mother.shape[0]}) must match number of individuals ({n_ind})."
        assert self.father.shape[0] == n_ind, f"Father array length ({self.father.shape[0]}) must match number of individuals ({n_ind})."
        assert self.sex.shape[0] == n_ind, f"Sex array length ({self.sex.shape[0]}) must match number of individuals ({n_ind})."
        
        # Validate trait-related array shapes
        assert self.pheno.shape[0] == n_ind, f"Phenotype (pheno) array length ({self.pheno.shape[0]}) must match number of individuals ({n_ind})."
        assert self.fixEff.shape[0] == n_ind, f"Fixed effect (fixEff) array length ({self.fixEff.shape[0]}) must match number of individuals ({n_ind})."
        
        # Validate optional genetic value components
        if self.bv is not None:
            assert self.bv.shape[0] == n_ind, f"Breeding value (bv) array length ({self.bv.shape[0]}) must match number of individuals ({n_ind})."
        if self.dd is not None:
            assert self.dd.shape[0] == n_ind, f"Dominance deviation (dd) array length ({self.dd.shape[0]}) must match number of individuals ({n_ind})."
        if self.aa is not None:
            assert self.aa.shape[0] == n_ind, f"Epistatic deviation (aa) array length ({self.aa.shape[0]}) must match number of individuals ({n_ind})."
        if self.ebv is not None:
            assert self.ebv.shape[0] == n_ind, f"EBV array length ({self.ebv.shape[0]}) must match number of individuals ({n_ind})."


    @property
    def nInd(self) -> int:
        """Returns the number of individuals in the population."""
        return self.geno.shape[0]
    
    @property
    def nChr(self) -> int:
        """Returns the number of chromosomes in the population."""
        return self.geno.shape[1]

    @property
    def nTraits(self) -> int:
        """Returns the number of traits, inferred from the breeding value (bv) shape."""
        if self.bv is None or self.bv.ndim <= 1:
            return 0
        return self.bv.shape[1]
    
    @property
    def haplo_matrix(self) -> jnp.ndarray:
        """ returns a haplotype matrix of shape (n_ind*ploidy, n_chr*n_markers)"""
        return self.geno.transpose(0, 2, 1, 3).reshape(self.geno.shape[0] * self.geno.shape[2], -1)

    @property
    def dosage(self) -> jnp.ndarray:
        """
        Calculates the dosage of alternate alleles for each individual.

        The dosage is the sum of alleles across the ploidy dimension, resulting
        in a 2D matrix where each entry represents the count of the alternate
        allele at a specific locus for an individual.

        Returns:
            A JAX array of shape `(nInd, nLoci)`, where nLoci is the total
            number of loci across all chromosomes.
        """
        # Sum over the ploidy axis (axis=2) to get dosage per chromosome
        # Shape: (nInd, nChr, nLoci_per_chr)
        dosage_per_chr = jnp.sum(self.geno, axis=2)

        # Reshape to combine the chromosome and loci dimensions
        # Shape: (nInd, nChr * nLoci_per_chr)
        return dosage_per_chr.reshape(self.nInd, -1)

    def __repr__(self) -> str:
        """Provides a concise representation of the Population object."""
        return (f"Population(nInd={self.nInd}, nTraits={self.nTraits}, "
                f"has_ebv={'Yes' if self.ebv is not None else 'No'})")    


In [19]:
show_doc(Population)

---

[source](https://github.com/cjGO/chewc/blob/main/chewc/population.py#L36){target="_blank" style="float:right; font-size:smaller"}

### Population

>      Population (geno:jax.Array, id:jax.Array, iid:jax.Array,
>                  mother:jax.Array, father:jax.Array, sex:jax.Array,
>                  pheno:jax.Array, fixEff:jax.Array,
>                  gv:Optional[jax.Array]=None, bv:Optional[jax.Array]=None,
>                  dd:Optional[jax.Array]=None, aa:Optional[jax.Array]=None,
>                  ebv:Optional[jax.Array]=None, gxe:Optional[jax.Array]=None,
>                  misc:Dict=<factory>, miscPop:Dict=<factory>)

*A container for all data related to a population of individuals, designed
for JAX-based genetic simulations.

This structure is immutable. All operations that modify a population should
return a new Population object.

Attributes:
    geno (jnp.ndarray): A 4D array representing the genotypes of the population.
        Shape: `(nInd, nChr, ploidy, nLoci)`. dtype: `jnp.uint8`.

    id (jnp.ndarray): The primary, user-facing identifier for each individual.
        These IDs may not be contiguous or sorted. Shape: `(nInd,)`.
    iid (jnp.ndarray): The internal, zero-indexed, contiguous identifier.
        Crucial for robust indexing in JAX operations. Shape: `(nInd,)`.
    mother (jnp.ndarray): Array of internal IDs (`iid`) for the mother of each
        individual. A value of -1 indicates no known mother. Shape: `(nInd,)`.
    father (jnp.ndarray): Array of internal IDs (`iid`) for the father of each
        individual. A value of -1 indicates no known father. Shape: `(nInd,)`.
    sex (jnp.ndarray): The sex of each individual, represented numerically
        (e.g., 0 for male, 1 for female). dtype: `jnp.int8`. Shape: `(nInd,)`.

    pheno (jnp.ndarray): The phenotypic values for each individual.
        Shape: `(nInd, nTraits)`.
    fixEff (jnp.ndarray): The value of a fixed effect for each individual,
        often used as an intercept in genomic selection models. Shape: `(nInd,)`.

    bv (Optional[jnp.ndarray]): The true breeding values (additive genetic effects)
        for each individual. Shape: `(nInd, nTraits)`.
    dd (Optional[jnp.ndarray]): The true dominance deviations for each individual.
        Shape: `(nInd, nTraits)`.
    aa (Optional[jnp.ndarray]): The true additive-by-additive epistatic deviations
        for each individual. Shape: `(nInd, nTraits)`.

    ebv (Optional[jnp.ndarray]): The estimated breeding values for each
        individual. Shape: `(nInd, nTraits)`.
    gxe (Optional[jnp.ndarray]): Genotype-by-environment interaction effects.
        Shape depends on the specific GxE model.

    misc (Dict): A dictionary for storing miscellaneous, non-JAX-critical
        metadata about individuals. Static.
    miscPop (Dict): A dictionary for storing miscellaneous, non-JAX-critical
        metadata about the entire population. Static.*

In [20]:
#| export


def quick_haplo(
    key: jax.random.PRNGKey, 
    n_ind: int, 
    n_chr: int, 
    n_loci_per_chr: int, 
    ploidy: int = 2, 
    inbred: bool = False,
    chr_len_cm: float = 100.0
) -> Tuple[Population, jnp.ndarray]:
    """
    Creates a new population with random haplotypes and a uniform genetic map.

    This function is a self-contained founder population generator, analogous to
    AlphaSimR's `quickHaplo`. It no longer depends on a SimParam object.
    Instead, it returns both the Population and the genetic map needed to
    construct a SimParam object later.

    Args:
        key: A JAX random key for reproducibility.
        n_ind: Number of individuals to create.
        n_chr: Number of chromosomes.
        n_loci_per_chr: Number of loci on each chromosome.
        ploidy: The ploidy level of the individuals (default: 2).
        inbred: If True, individuals will be fully inbred (homozygous).
        chr_len_cm: The length of each chromosome in centiMorgans for the
                    generated uniform genetic map (default: 100.0).

    Returns:
        A tuple containing:
        - A new Population object with random founder individuals.
        - A JAX array representing the genetic map, with shape 
          `(n_chr, n_loci_per_chr)`.
    """
    key, geno_key, sex_key = jax.random.split(key, 3)

    # Generate random haplotypes for all individuals and chromosomes
    # Shape: (nInd, nChr, ploidy, nLoci)
    if inbred:
        # Generate one set of haplotypes and tile it across the ploidy axis
        base_haplotypes = jax.random.randint(geno_key, (n_ind, n_chr, 1, n_loci_per_chr), 0, 2, dtype=jnp.uint8)
        geno = jnp.tile(base_haplotypes, (1, 1, ploidy, 1))
    else:
        # Fully random, outbred individuals
        geno = jax.random.randint(geno_key, (n_ind, n_chr, ploidy, n_loci_per_chr), 0, 2, dtype=jnp.uint8)

    # --- Create Pedigree and IDs using JAX arrays ---
    ids = jnp.arange(n_ind)
    sex_array = jax.random.choice(sex_key, jnp.array([0, 1], dtype=jnp.int8), (n_ind,))

    # --- Generate a uniform genetic map ---
    # Each chromosome has loci evenly spaced from 0 to chr_len_cm
    loci_pos = jnp.linspace(0., chr_len_cm, n_loci_per_chr)
    genetic_map = jnp.tile(loci_pos, (n_chr, 1))

    population = Population(
        geno=geno,
        id=ids,
        iid=ids,  # In a new pop, id and iid are the same
        mother=jnp.full(n_ind, -1, dtype=jnp.int32),
        father=jnp.full(n_ind, -1, dtype=jnp.int32),
        sex=sex_array,
        pheno=jnp.zeros((n_ind, 0)),
        fixEff=jnp.zeros(n_ind, dtype=jnp.float32), # Default fixed effect of 0
        bv=jnp.zeros((n_ind, 0)),  # No traits by default
        dd=None,
        aa=None,
    )
    
    return population, genetic_map

In [21]:
#| test

# 1. ARRANGE: Set up the parameters for the test
key = jax.random.PRNGKey(42)
n_ind = 10
n_chr = 2
n_loci = 5
ploidy = 2

# 2. ACT: Call the refactored function
test_pop, test_gen_map = quick_haplo(
    key, 
    n_ind=n_ind, 
    n_chr=n_chr, 
    n_loci_per_chr=n_loci, 
    ploidy=ploidy, 
    inbred=False
)

# 3. ASSERT: Check that the results are correct

def test_quick_haplo_output_types():
    "Test that quick_haplo returns the correct types."
    assert isinstance(test_pop, Population)
    assert isinstance(test_gen_map, jnp.ndarray)

def test_quick_haplo_dimensions():
    "Test if the created population and map have the correct dimensions."
    assert test_pop.nInd == n_ind
    assert test_pop.nChr == n_chr
    assert test_pop.geno.shape == (n_ind, n_chr, ploidy, n_loci)
    assert test_gen_map.shape == (n_chr, n_loci)

def test_quick_haplo_gen_map_values():
    "Test if the genetic map values are correct."
    # For n_loci=5 and chr_len_cm=100.0, positions should be [0., 25., 50., 75., 100.]
    expected_positions = jnp.linspace(0., 100., n_loci)
    assert jnp.allclose(test_gen_map[0, :], expected_positions) # Check first chromosome
    assert jnp.allclose(test_gen_map[1, :], expected_positions) # Check second chromosome

def test_quick_haplo_inbred_flag():
    "Test the `inbred` flag functionality."
    key_inbred = jax.random.PRNGKey(43)
    inbred_pop, _ = quick_haplo(
        key_inbred, 
        n_ind=n_ind, 
        n_chr=n_chr, 
        n_loci_per_chr=n_loci, 
        ploidy=ploidy, 
        inbred=True
    )
    
    # For an inbred individual, all haplotype copies should be identical
    first_ind_geno = inbred_pop.geno[0] # Shape: (nChr, ploidy, nLoci)
    # Check that all ploidy copies are identical to the first copy
    for p in range(1, ploidy):
        assert jnp.array_equal(first_ind_geno[:, 0, :], first_ind_geno[:, p, :])

In [ ]:
#| export

def msprime_pop(
    key: jax.random.PRNGKey,
    n_ind: int,
    n_loci_per_chr: int,
    n_chr: int,
    ploidy: int = 2,
    effective_population_size: int = 1_000,  # Reduced default from 10_000
    mutation_rate: float = 2e-8,
    recombination_rate_per_chr: float = 2e-8,
    maf_threshold: float = 0.1,
    num_simulated_individuals: int = None,  # Now calculated dynamically
    base_chr_length: int = 1_000_000
) -> Tuple[Population, jnp.ndarray]:
    """
    Creates a new founder population using msprime coalescent simulation.

    Generates genotypes and a genetic map based on population genetics principles.
    Updated with improved parameter validation and more reasonable defaults.

    Args:
        key: JAX random key.
        n_ind: Number of founder individuals to generate.
        n_loci_per_chr: Number of SNPs (loci) to select per chromosome.
        n_chr: Number of chromosomes.
        ploidy: The ploidy level of the individuals (default: 2).
        effective_population_size: The effective population size for simulation.
            Reduced default to 1,000 for better performance and memory usage.
            Larger values (>50,000) may cause memory issues.
        mutation_rate: The mutation rate for the simulation.
        recombination_rate_per_chr: Recombination rate per chromosome.
        maf_threshold: Minimum allele frequency threshold for SNPs.
        num_simulated_individuals: Number of individuals to simulate initially.
            If None, will be set to max(n_ind * 2, 1000) for better variant diversity.
        base_chr_length: Length of each chromosome in base pairs.

    Returns:
        A tuple containing:
        - A new Population object with random founder individuals.
        - A JAX array representing the genetic map, with shape 
          `(n_chr, n_loci_per_chr)`.

    Raises:
        ValueError: If parameters are invalid or likely to cause memory issues.
    """
    # --- Parameter Validation ---
    if effective_population_size > 100_000:
        raise ValueError(
            f"Effective population size {effective_population_size} is too large and may cause "
            f"memory issues. Consider using values <= 50,000. For very large populations, "
            f"consider using quick_haplo() instead."
        )
    
    if effective_population_size < 10:
        raise ValueError(
            f"Effective population size {effective_population_size} is too small. "
            f"Use values >= 10 for realistic simulations."
        )
    
    # Set num_simulated_individuals dynamically if not provided
    if num_simulated_individuals is None:
        # Ensure we have enough diversity by simulating more individuals than needed
        # But cap it to avoid memory issues
        num_simulated_individuals = min(max(n_ind * 2, 1000), 10_000)
    
    if n_ind > num_simulated_individuals:
        raise ValueError(
            f"Number of founders requested ({n_ind}) cannot exceed the base simulated "
            f"population size ({num_simulated_individuals})."
        )
    
    # Warn if parameters might cause performance issues
    total_genome_size = n_chr * base_chr_length
    if total_genome_size > 1e9:  # 1 billion bp
        import warnings
        warnings.warn(
            f"Large total genome size ({total_genome_size} bp) may cause slow simulation. "
            f"Consider reducing base_chr_length or n_chr for faster performance.",
            UserWarning
        )
    
    if num_simulated_individuals * ploidy > 50_000:
        import warnings
        warnings.warn(
            f"Large number of haplotypes ({num_simulated_individuals * ploidy}) may cause "
            f"memory issues. Consider reducing num_simulated_individuals.",
            UserWarning
        )

    # --- Derive Seeds ---
    key, seed_key, sex_key, numpy_seed_key = jax.random.split(key, 4)
    random_seed = int(jnp.sum(seed_key))
    numpy_seed = int(jnp.sum(numpy_seed_key))
    rng = default_rng(numpy_seed)

    # --- Chromosome Lengths ---
    chromosome_lengths = [base_chr_length] * n_chr

    # --- Run msprime Simulation ---
    num_haplotypes = num_simulated_individuals * ploidy

    # Create the recombination map for msprime
    rate_map_positions = [0] + list(np.cumsum(chromosome_lengths))
    rate_map_rates = [recombination_rate_per_chr] * len(chromosome_lengths)
    rate_map = msprime.RateMap(position=rate_map_positions, rate=rate_map_rates)

    try:
        ts = msprime.sim_ancestry(
            samples=num_haplotypes, 
            population_size=effective_population_size,
            recombination_rate=rate_map, 
            random_seed=random_seed
        )
        mts = msprime.sim_mutations(ts, rate=mutation_rate, random_seed=random_seed)
    except Exception as e:
        # Provide helpful error message for common issues
        if "memory" in str(e).lower() or "malloc" in str(e).lower():
            raise RuntimeError(
                f"Memory allocation failed during msprime simulation. This is likely due to "
                f"too large parameter combination. Try reducing effective_population_size "
                f"(current: {effective_population_size}), num_simulated_individuals "
                f"(current: {num_simulated_individuals}), or genome size. "
                f"Original error: {str(e)}"
            ) from e
        else:
            raise RuntimeError(f"msprime simulation failed: {str(e)}") from e

    # --- Data Extraction ---
    true_num_individuals = mts.num_samples // ploidy
    all_variants = list(mts.variants())

    # Initialize final data structures
    genetic_map = np.full((n_chr, n_loci_per_chr), np.nan)
    full_haplotype_matrix = np.full(
        (true_num_individuals, n_chr, ploidy, n_loci_per_chr), np.nan
    )

    for i in range(n_chr):
        chr_start, chr_end, recomb_rate = rate_map.left[i], rate_map.right[i], rate_map.rate[i]

        eligible_snps = [
            var for var in all_variants
            if chr_start <= var.site.position < chr_end 
            and len(var.alleles) == 2 
            and min(np.mean(var.genotypes), 1 - np.mean(var.genotypes)) > maf_threshold
        ]

        num_found = len(eligible_snps)
        num_to_select = min(num_found, n_loci_per_chr)

        if num_to_select > 0:
            selected_indices = rng.choice(len(eligible_snps), num_to_select, replace=False)
            selected_snps = [eligible_snps[i] for i in selected_indices]
            selected_snps.sort(key=lambda v: v.site.position)

            for snp_idx, snp in enumerate(selected_snps):
                genotypes_for_snp = snp.genotypes.reshape(true_num_individuals, ploidy)
                full_haplotype_matrix[:, i, :, snp_idx] = genotypes_for_snp

            positions_cm = [(v.site.position - chr_start) * recomb_rate * 100 for v in selected_snps]
            genetic_map[i, :num_to_select] = positions_cm
        elif num_found == 0:
            import warnings
            warnings.warn(
                f"No variants found for chromosome {i} with MAF > {maf_threshold}. "
                f"Consider lowering maf_threshold or increasing mutation_rate/effective_population_size.",
                UserWarning
            )

    # --- Sample Founders and Format for Population Object ---
    founder_indices = np.sort(rng.choice(true_num_individuals, n_ind, replace=False))
    founder_haplotypes = full_haplotype_matrix[founder_indices, :, :, :]

    # Convert to JAX arrays
    geno = jnp.array(founder_haplotypes, dtype=jnp.uint8)
    gen_map_jax = jnp.array(genetic_map)

    # --- Create Pedigree and IDs using JAX arrays ---
    ids = jnp.arange(n_ind)
    sex_array = jax.random.choice(sex_key, jnp.array([0, 1], dtype=jnp.int8), (n_ind,))

    pop = Population(
        geno=geno,
        id=ids,
        iid=ids,
        mother=jnp.full(n_ind, -1, dtype=jnp.int32),
        father=jnp.full(n_ind, -1, dtype=jnp.int32),
        sex=sex_array,
        pheno=jnp.zeros((n_ind, 0)),
        fixEff=jnp.zeros(n_ind, dtype=jnp.float32),
        bv=jnp.zeros((n_ind, 0)),
        dd=None,
        aa=None,
        miscPop={
            'msprime_params': {
                'effective_population_size': effective_population_size,
                'mutation_rate': mutation_rate,
                'recombination_rate_per_chr': recombination_rate_per_chr,
                'maf_threshold': maf_threshold,
                'num_simulated_individuals': num_simulated_individuals,
                'base_chr_length': base_chr_length
            }
        }
    )

    return pop, gen_map_jax

In [23]:
#| test


# 1. ARRANGE: Define test parameters
key = jax.random.PRNGKey(123)
n_ind = 10
n_loci_per_chr = 5
n_chr = 2
ploidy = 2

# 2. ACT: Call the function and unpack the tuple
test_pop, test_gen_map = msprime_pop(
    key,
    n_ind,
    n_loci_per_chr,
    n_chr,
    ploidy
)

# 3. ASSERT: Perform tests

def test_msprime_pop_output_type():
    "Test that msprime_pop returns a Population object and a JAX array."
    assert isinstance(test_pop, Population)
    assert isinstance(test_gen_map, jnp.ndarray)

def test_msprime_pop_dimensions():
    "Test if the created population has the correct dimensions."
    test_eq(test_pop.nInd, n_ind)
    test_eq(test_pop.nChr, n_chr)
    test_eq(test_pop.geno.shape, (n_ind, n_chr, ploidy, n_loci_per_chr))

def test_msprime_pop_genetic_map_dimensions():
    "Test that the returned genetic map has the correct dimensions."
    test_eq(test_gen_map.shape, (n_chr, n_loci_per_chr))

def test_msprime_pop_not_all_zeros():
    "Test that the genotypes are not all zeros (some variation exists)."
    assert jnp.any(test_pop.geno != 0)

def test_msprime_pop_different_key():
    "Test that a different key produces a different population."
    key2 = jax.random.PRNGKey(456)
    test_pop2, _ = msprime_pop(
        key2,
        n_ind,
        n_loci_per_chr,
        n_chr,
        ploidy
    )
    test_ne(jnp.sum(test_pop.geno), jnp.sum(test_pop2.geno))

def test_msprime_pop_reproducibility():
    "Test that the same key produces the exact same population and map."
    key1 = jax.random.PRNGKey(789)
    pop1, map1 = msprime_pop(key1, n_ind, n_loci_per_chr, n_chr, ploidy)
    pop2, map2 = msprime_pop(key1, n_ind, n_loci_per_chr, n_chr, ploidy)
    assert jnp.array_equal(pop1.geno, pop2.geno)
    assert jnp.allclose(map1, map2, equal_nan=True)

In [24]:
#| hide
import nbdev; nbdev.nbdev_export()